# Challenge #2 - Make a New Forecasting Function

<small>Python for Data Science Automation Course (DS4B 101-P)</small><br>
<small>Business Science</small> 

## Challenge Summary

This is a short challenge to extend your ability to review python `sktime` documentation. You will go through a series of questions related to:

1. Creating a New Forecasting Function using Exponential Smoothing

2. Forecasting Revenue By Customers

### Data Science Request

Your Data Science Manager has requested you to implement a second forecasting algorithm called "Exponential Smoothing" or ETS. Your goal is to leverage your knowledge of the existing ARIMA function you've created and make a new function that swaps ARIMA for ETS.  

### Objectives

1. Use your `sktime` documentation sloothing skills to make a a new ETS forecasting function.

2. Apply the function to forecast revenue by customers.

## Getting Started

### Check Working Directory

To read in the data, make sure your current working directory is set to the project directory. Two useful [jupyter magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) are:

1. `%pwd`: [Print working directory](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-cd) (you can detect your current directory)
2. `%cd`: You can [change directory](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-cd) to your working directory using relative paths or full paths. 


In [15]:
%pwd

'/Users/vadiveld/PycharmProjects/Python-Business-science/Challenge 1 2'

### Python Package Imports

In [13]:
import pandas as pd
import numpy as np

from path import Path

from transformer.bike_order_transformer import BikeOrderTransformer
from my_pandas_extension.timeseries_func import summarize_by_time

database_folder_path = Path("../data/database")

conn_string = f"sqlite:///{database_folder_path}/bikes_order_database.sqlite"

bike_order_line_df = BikeOrderTransformer(conn_string).transform_data()

bike_order_line_df["order_date"] = pd.to_datetime(bike_order_line_df["order_date"])

ModuleNotFoundError: No module named 'my_pandas_extensions'

In [12]:
bike_order_line_df.head()

NameError: name 'bike_order_line_df' is not defined

### Import Data

We'll make use of our `arima_forecast()` function as a starting point for modification. 

In [5]:
from sktime.forecasting.ets import AutoETS
from tqdm import tqdm

from pydantic import (
    validate_call,
    StrictFloat,
    ConfigDict,
    Field,
    StrictInt,
)
from typing import Literal, Annotated, Optional


@validate_call(config=ConfigDict(arbitrary_types_allowed=True))
def ets_forecast(
        data: pd.DataFrame,
        h: Annotated[int, Field(strict=True, gt=0)],
        sp: Literal[3, 6, 12, 24],
        seasonal: Optional[Literal['add', 'null']] = None,
        trend: Optional[Literal['add', 'null']] = None,
        alpha: Annotated[StrictFloat, Field(strict=True, gt=0, lt=1)] = 0.95,
        suppress_warmings=True,
        *args,
        **kwargs):
    # Checks

    # Handle Inputs ----
    df = data

    # FOR LOOP ----

    model_results_dict = {}
    for col in tqdm(df.columns, mininterval=0):
        # Series Extraction
        y = df[col]

        # Modeling
        forecaster = AutoETS(
                sp=sp,
                seasonal=seasonal,
                trend=trend,
                *args,
                **kwargs
        )

        forecaster.fit(y)

        predictions = forecaster.predict(fh=np.arange(1, h + 1))
        predictions_interval = forecaster.predict_interval(coverage=alpha)
        predictions_interval.index = predictions.index

        predictions_return_df = pd.concat([y, predictions, predictions_interval], axis=1)

        predictions_return_df = pd.concat([y, predictions, predictions_interval], axis=1)

        predictions_return_df.columns = ["value", "prediction", "ci_lo", "ci_hi"]

        model_results_dict[col] = predictions_return_df

    # Stack Each Dict Element on Top of Each Other
    model_results_df = pd.concat(
            model_results_dict,
            axis=0
    )

    # Handle Names
    nms = [*df.columns.names, *df.index.names]
    model_results_df.index.names = nms

    # Reset Index
    ret = model_results_df.reset_index()

    # Drop columns containing "level"
    cols_to_keep = ~ret.columns.str.startswith("level_")

    ret = ret.iloc[:, cols_to_keep]

    return ret

## Challenge

Perform the following tasks. 

### Task 1: Make an ETS Forecasting Function

1. Copy the `arima_forecast`
2. Import the [AutoETS Function](https://www.sktime.org/en/v0.5.3/api_reference/modules/auto_generated/sktime.forecasting.ets.AutoETS.html?highlight=autoets) from **version 0.5.3** of `sktime`
3. Rename the `arima_forecast()` function to `ets_forecast()`
4. Change the internals of the function to swap out `AutoARIMA()` for `AutoETS()`

### Task 2: Test the `ets_forecast()` function

In [ ]:
bike_sales_cat2_m_df = bike_order_line_df.summarize_by_time(  # type: ignore
        date_column="order_date",
        value_column=["total_price"],
        groups=["category_2"],
        rules="MS",
        time_format="period",
)

bike_sales_cat2_m_df.head()

1. Start with `bike_sales_cat2_m_df`
2. Use the `ets_forecast()` function to forecast with the following parameters:
```
ets_forecast(
    data     = bike_sales_cat2_m_df,
    h        = 12, 
    sp       = 12, 
    seasonal = "add", # For additive seasonality 
    trend    = "add"  # For additive trend
)
```
3. Troubleshoot the Prediction "Not Yet Implemented Error" 
    - Go back to your function and make changes to remove the confidence interval
    - Remove the `alpha` argument from the function
4. Use `.groupby()` to group by `category_2`.
5. Use `.plot()` to plot with `x = 'order_date'.

In [ ]:
# Code

fcast = ets_forecast(
        data=bike_sales_cat2_m_df,
        h=12,
        sp=12,
        seasonal="add",  # For additive seasonality
        trend="add"  # For additive trend
)


In [ ]:
fcast.groupby('category_2').plot(x='order_date')

## Nice Work

This challenge tested your ability to research documentatoin and debug code. If you nailed it, you're doing awesome. If you didn't, guess what - You're still awesome too! Keep in mind you're learning, it takes time, and these challenges are meant to be *challenging*. 